In [1]:
import tensorflow as tf
import utils
import numpy as np
import random as rn
import os

In [152]:
SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
rn.seed(SEED)

data = np.load(os.path.join('.','data','royalty.npz'))

triples = data['triples']
traces = data['traces']
entities = data['entities'].tolist()
num_entities = len(entities)
relations = data['relations'].tolist()
num_relations = len(relations)
embedding_dim = 3
ent2idx = dict(zip(entities, range(num_entities)))
rel2idx = dict(zip(relations, range(num_relations)))

In [153]:
from tensorflow.keras.layers import Input, Embedding
from tensorflow.keras.initializers import RandomUniform

entity_embeddings = Embedding(
    input_dim=num_entities,
    output_dim=embedding_dim,
    name='entity_embeddings',
    embeddings_initializer=RandomUniform(
        minval=-1,
        maxval=1,
        seed=SEED
        )
    )

relation_embeddings = Embedding(
    input_dim=num_relations,
    output_dim=embedding_dim,
    name='relation_embeddings',
    embeddings_initializer=RandomUniform(
        minval=-1,
        maxval=1,
        seed=SEED
        )
    )

In [173]:
import tensorflow.keras.backend as K

# class RGCN(tf.keras.layers.Layer):
    
#     def __init__(self,num_relations,output_dim,**kwargs):
#         super(RGCN,self).__init__(**kwargs)
#         self.num_relations = num_relations
#         self.output_dim = output_dim
    
#     def build(self,input_shape):
        
#         in_shape = input_shape[-1][-1]
#         self.kernel = self.add_weight(
#             shape=(self.num_relations,self.output_dim,in_shape),
#             trainable=True,
#             initializer="random_normal"
#         )
    
#     def call(self,inputs):
        
#         head_e, tail_e = inputs

#         head_mat = self.forward_pass(head_e)
#         tail_mat = self.forward_pass(tail_e)
        
#         return head_mat,tail_mat

#     def forward_pass(self,batch_embeddings):
        
#         embedding_mat = K.dot(self.kernel,K.transpose(batch_embeddings))
        
#         return K.transpose(K.sum(embedding_mat, axis=0))
class RGCN(tf.keras.layers.Layer):
    def __init__(self,num_relations,output_dim,**kwargs):
        super(RGCN,self).__init__(**kwargs)
        self.num_relations = num_relations
        self.output_dim = output_dim
    
    def build(self,input_shape):

        in_shape = input_shape[-1][-1]
        self.kernel = self.add_weight(
            shape=(self.num_relations,self.output_dim,in_shape),
            trainable=True,
            initializer="random_normal"
        )
        
    def call(self,inputs):
        return inputs#K.sum(K.dot(self.kernel,K.transpose(inputs)), axis=2)

In [143]:
#np.ones((1,3,3))*np.eye(3)

In [210]:
head_input = tf.keras.Input(shape=(), name='head_input',dtype=tf.int64)
rel_input = tf.keras.Input(shape=(), name='rel_input',dtype=tf.int64)
tail_input = tf.keras.Input(shape=(), name='tail_input',dtype=tf.int64)
all_entities = tf.keras.Input(shape=(),name='all_input',dtype=tf.int64)

In [186]:
head_e = entity_embeddings(head_input)
rel_e = relation_embeddings(rel_input)
tail_e = entity_embeddings(tail_input)
all_e = entity_embeddings(all_entities)

In [171]:
#RGCN(num_relations,1)(RGCN(num_relations,5)([head_e,tail_e]))
RGCN(num_relations,5)([all_e])

[<tf.Tensor 'rgcn_63/rgcn_63/Identity:0' shape=(None, 3) dtype=float32>]

In [102]:
#np.dot(np.ones((4,5,3)),np.zeros((3,100))).shape#.sum(axis=0).T.shape#.sum(axis=(2)).shape 

In [222]:
class DistMult(tf.keras.layers.Layer):
    def __init__(self, num_relations,embedding_dim):
        super(DistMult,self).__init__()
        self.num_relations = num_relations
        self.embedding_dim = embedding_dim
        
    def build(self,input_shape):
        self.kernel = self.add_weight(
            shape=(self.num_relations,self.embedding_dim),
            trainable=True,
            initializer='random_normal'
        )
    def call(self,inputs):
        #must be of dimensions (batch size,)
        embeddings,head_idx,rel_idx, tail_idx = inputs
        
        updated_head_e = embedding_ops.embedding_lookup_v2(embeddings,head_idx)
        rel_e = embedding_ops.embedding_lookup_v2(self.kernel,rel_idx)
        updated_tail_e = embedding_ops.embedding_lookup_v2(embeddings,tail_idx)

        return K.sum(updated_head_e*rel_e*updated_tail_e, axis=-1)

In [220]:
# def get_adj_and_degrees(num_nodes, triplets):
#     """ Get adjacency list and degrees of the graph
#     """
#     adj_list = [[] for _ in range(num_nodes)]
#     for i,triplet in enumerate(triplets):
#         adj_list[triplet[0]].append([i, triplet[2]])
#         adj_list[triplet[2]].append([i, triplet[0]])

#     degrees = np.array([len(a) for a in adj_list])
#     adj_list = [np.array(a) for a in adj_list]
#     return adj_list, degrees
# get_adj_and_degrees(num_entities,utils.array2idx(triples, ent2idx,rel2idx))

#use custom weight initializer -> set new embedding layer with embeddings from RGCN
#feed new embedding layer into distmult layer

In [223]:
#DistMult(num_relations,embedding_dim)([RGCN(num_relations,5)([all_e]),head_e,rel_input,tail_e])
from tensorflow.python.ops import embedding_ops
DistMult(num_relations,embedding_dim)([RGCN(num_relations,5)([all_e]),head_input, rel_input,tail_input])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


<tf.Tensor 'dist_mult_19/Sum:0' shape=(None,) dtype=float32>